In [72]:
import csv
import copy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import pi
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score
from sklearn.svm import SVR
from sklearn.svm import SVC


In [55]:
header = None
features = None
with open('../feature_process/aligned_features_sorted.csv' ,'r')as f:
    features = csv.reader(f)
    features = list(features)
    header = features[0]
    features = features[1:]
    

In [56]:
len(features)

2999

In [76]:
# top features paramter allow users to change features 
def split_train_test(lower,upper,data,top_features = 33): # fips code upper and lower bound for a state. ex. wisconsin 55000,56000
    data = copy.deepcopy(data)
    training = []
    testing =[]
    fips_train = []
    fips_test = []
    for row in data:
        if lower<= int(row[0]) < upper:
            testing.append(row[1:(top_features+1)])
            fips_test.append(row[0])
        else:
            training.append(row[1:(top_features+1)])
            fips_train.append(row[0])
    for i in range(len(training)):
        for j in range(len(training[0])):
            training[i][j] = float(training[i][j])
    for i in range(len(testing)):
        for j in range(len(testing[0])):
            testing[i][j] = float(testing[i][j])        
    return training, testing,fips_train,fips_test

In [77]:
train_x,test_x,fips_train,fips_test = split_train_test(55000,56000,features)

In [78]:
len(train_x[0])

32

# Continuous Problem (increase rate)

##  k = 5  Cross Validation Error

In [91]:
labels = None
k = 5
file_name = '../label_process/labels/{}_increase_{}'
data_type = ['deaths','cases']
interval_num = ['1','5','10']
for d_type in data_type:
    for i_num in interval_num:
        with open(file_name.format(d_type,i_num),'r') as f:
            labels = csv.reader(f)
            labels = list(labels)
        
    
        train_y,test_y,fips_train,fips_test = split_train_test(55000,56000,labels)
        #  cross validation split train test k = 5
        interval_length = int(len(train_x)/k)
        tr_set = []
        tr_labels = []
        va_set = []
        va_labels = []
        total_mse = []
        svm_mse = []
        training_set = copy.deepcopy(train_x)
        training_labels = copy.deepcopy(train_y)
        for kv_idx in range(k):
            va_set = training_set[interval_length * kv_idx:interval_length*(kv_idx+1)]
            va_labels = training_labels[interval_length * kv_idx:interval_length*(kv_idx+1)]
            tr_set = training_set[:interval_length * kv_idx] + training_set[interval_length*(kv_idx+1):]
            tr_labels = training_labels[:interval_length * kv_idx] + training_labels[interval_length*(kv_idx+1):]
            
            va_label_df = pd.DataFrame(va_labels,dtype = float)
            tr_label_df = pd.DataFrame(tr_labels,dtype = float)
            
            va_prediction = []
            svr_va_prediction = []
            va_answer = []
            for time_zone in range(int(float(i_num))):
                tr_y = list(tr_label_df.loc[:,time_zone])
                va_y = list(va_label_df.loc[:,time_zone])
                regr = RandomForestRegressor()
                svr_regr = SVR()
                regr.fit(tr_set, tr_y)
                svr_regr.fit(tr_set, tr_y)
                va_prediction = va_prediction + list(regr.predict(va_set))
                svr_va_prediction = svr_va_prediction + list(svr_regr.predict(va_set))
                va_answer += va_y
        
            total_mse.append(mean_absolute_error(va_answer,va_prediction))
            svm_mse.append(mean_absolute_error(va_answer,svr_va_prediction))
        print('MAE for N = {}, {} increase rates, random forest'.format(i_num,d_type))
        print(sum(total_mse)/len(total_mse))
        print('MAE for N = {}, {} increase rates, SVR'.format(i_num,d_type))
        print(sum(svm_mse)/len(svm_mse))


MAPE for N = 1, deaths increase rates, random forest
0.00039258050013288624
MAPE for N = 1, deaths increase rates, SVR
0.002806304550705679
MAPE for N = 5, deaths increase rates, random forest
0.00012156162799185823
MAPE for N = 5, deaths increase rates, SVR
0.0012325553659813322
MAPE for N = 10, deaths increase rates, random forest
7.115228389125938e-05
MAPE for N = 10, deaths increase rates, SVR
0.000728304645987609
MAPE for N = 1, cases increase rates, random forest
0.012664492404519685
MAPE for N = 1, cases increase rates, SVR
0.06209782592115588
MAPE for N = 5, cases increase rates, random forest
0.0036503566204986945
MAPE for N = 5, cases increase rates, SVR
0.0454246680474594
MAPE for N = 10, cases increase rates, random forest
0.0020570714046789225
MAPE for N = 10, cases increase rates, SVR
0.029007918521587205


## testing error (Random Forest/SVM)

In [62]:
labels = None
file_name = '../label_process/labels/{}_increase_{}'
data_type = ['deaths','cases']
interval_num = ['1','5','10']
for d_type in data_type:
    for i_num in interval_num:
        with open(file_name.format(d_type,i_num),'r') as f:
            labels = csv.reader(f)
            labels = list(labels)
            if len(labels) == (len(features)+1): ## excluding header if there is
                labels = labels[1:]
        
        train_y,test_y,fips_train,fips_test = split_train_test(55000,56000,labels)

        tr_set = []
        tr_labels = []
        va_set = []
        va_labels = []
        total_mse = []
        svm_mse = []
        training_set = copy.deepcopy(train_x)
        training_labels = copy.deepcopy(train_y)
        tr_label_df = pd.DataFrame(train_y)
        va_label_df = pd.DataFrame(test_y)
        
        va_prediction = []
        svr_va_prediction = []
        va_answer = []
        for time_zone in range(int(float(i_num))):
            tr_y = list(tr_label_df.loc[:,time_zone])
            va_y = list(va_label_df.loc[:,time_zone])
            regr = RandomForestRegressor()
            svr_regr = SVR()
            regr.fit(train_x, tr_y)
            svr_regr.fit(train_x, tr_y)
            va_prediction = va_prediction + list(regr.predict(test_x))
            svr_va_prediction = svr_va_prediction + list(svr_regr.predict(test_x))
            va_answer += va_y

        total_mse.append(mean_absolute_error(va_answer,va_prediction))
        svm_mse.append(mean_absolute_error(va_answer,svr_va_prediction))
        print('MAE for N = {}, {} increase rates, random forest'.format(i_num,d_type))
        print(sum(total_mse)/len(total_mse))
        print('MAE for N = {}, {} increase rates, SVR'.format(i_num,d_type))
        print(sum(svm_mse)/len(svm_mse))


MAPE for N = 1, deaths increase rates, random forest
0.0002961636956821826
MAPE for N = 1, deaths increase rates, SVR
0.0030582337692961637
MAPE for N = 5, deaths increase rates, random forest
8.802277105993334e-05
MAPE for N = 5, deaths increase rates, SVR
0.0012672066744095131
MAPE for N = 10, deaths increase rates, random forest
5.097310475719473e-05
MAPE for N = 10, deaths increase rates, SVR
0.0007445946470889617
MAPE for N = 1, cases increase rates, random forest
0.015550907630249574
MAPE for N = 1, cases increase rates, SVR
0.036717001001727576
MAPE for N = 5, cases increase rates, random forest
0.004706894720148842
MAPE for N = 5, cases increase rates, SVR
0.04501633457449545
MAPE for N = 10, cases increase rates, random forest
0.0025085474098432265
MAPE for N = 10, cases increase rates, SVR
0.029492560625535333


## testing error (linear regression)

In [79]:
labels = None
file_name = '../label_process/labels/{}_increase_{}'
data_type = ['deaths','cases']
interval_num = ['1','5','10']
for d_type in data_type:
    for i_num in interval_num:
        with open(file_name.format(d_type,i_num),'r') as f:
            labels = csv.reader(f)
            labels = list(labels)
            if len(labels) == (len(features)+1): ## excluding header if there is
                labels = labels[1:]
        
        train_y,test_y,fips_train,fips_test = split_train_test(55000,56000,labels)

        tr_set = []
        tr_labels = []
        va_set = []
        va_labels = []
        total_mse = []
        svm_mse = []
        training_set = copy.deepcopy(train_x)
        training_labels = copy.deepcopy(train_y)
        tr_label_df = pd.DataFrame(train_y)
        va_label_df = pd.DataFrame(test_y)
        
        va_prediction = []
        svr_va_prediction = []
        va_answer = []
        for time_zone in range(int(float(i_num))):
            tr_y = list(tr_label_df.loc[:,time_zone])
            va_y = list(va_label_df.loc[:,time_zone])
            regr = LinearRegression()
            svr_regr = SVR()
            regr.fit(train_x, tr_y)
            svr_regr.fit(train_x, tr_y)
            va_prediction = va_prediction + list(regr.predict(test_x))
            svr_va_prediction = svr_va_prediction + list(svr_regr.predict(test_x))
            va_answer += va_y

        total_mse.append(mean_absolute_error(va_answer,va_prediction))
        svm_mse.append(mean_absolute_error(va_answer,svr_va_prediction))
        print('MAE for N = {}, {} increase rates, linear regression'.format(i_num,d_type))
        print(sum(total_mse)/len(total_mse))
        #print('MAPE for N = {}, {} increase rates, SVR'.format(i_num,d_type))
        #print(sum(svm_mse)/len(svm_mse))



MAE for N = 1, deaths increase rates, linear regression
0.0002578075074066752
MAE for N = 5, deaths increase rates, linear regression
8.22592684093128e-05
MAE for N = 10, deaths increase rates, linear regression
4.6564476462268724e-05
MAE for N = 1, cases increase rates, linear regression
0.017813254536689346
MAE for N = 5, cases increase rates, linear regression
0.004983150523837751
MAE for N = 10, cases increase rates, linear regression
0.0027065228878186917


# Classification Problem

# Testing Accuracy(Random Forest/ SVM)

In [60]:
labels = None
look = []
answer_look = []
file_name = '../label_process/labels/{}_seriousness_{}'
data_type = ['deaths','cases']
interval_num = ['1']
for d_type in data_type:
    for i_num in interval_num:
        with open(file_name.format(d_type,i_num),'r') as f:
            labels = csv.reader(f)
            labels = list(labels)
            if len(labels) == (len(features)+1): ## excluding header if there is
                labels = labels[1:]
        
        train_y,test_y,fips_train,fips_test = split_train_test(55000,56000,labels)

        tr_set = []
        tr_labels = []
        va_set = []
        va_labels = []
        total_mse = []
        svm_mse = []
        training_set = copy.deepcopy(train_x)
        training_labels = copy.deepcopy(train_y)
        tr_label_df = pd.DataFrame(train_y)
        va_label_df = pd.DataFrame(test_y)
        va_prediction = []
        svr_va_prediction = []
        va_answer = []
        for time_zone in range(int(float(i_num))):
            tr_y = list(tr_label_df.loc[:,time_zone])
            va_y = list(va_label_df.loc[:,time_zone])
            regr = RandomForestClassifier()
            svr_regr = SVC(kernel = 'sigmoid',gamma='auto')
            regr.fit(train_x, tr_y)
            svr_regr.fit(train_x, tr_y)
            va_prediction = va_prediction + list(regr.predict(test_x))
            svr_va_prediction = svr_va_prediction + list(svr_regr.predict(test_x))
            va_answer += va_y
        look.append(va_prediction)
        look.append(svr_va_prediction)
        answer_look.append(va_answer)
        total_mse.append(accuracy_score(va_answer,va_prediction))
        svm_mse.append(accuracy_score(va_answer,svr_va_prediction))
        print('Accuracy for N = {}, {} seriousness, random forest'.format(i_num,d_type))
        print(sum(total_mse)/len(total_mse))
        print('Accuracy for N = {}, {} seriousness, SVR'.format(i_num,d_type))
        print(sum(svm_mse)/len(svm_mse))

Accuracy for N = 1, deaths seriousness, random forest
0.8055555555555556
Accuracy for N = 1, deaths seriousness, SVR
0.8611111111111112
Accuracy for N = 1, cases seriousness, random forest
0.8333333333333334
Accuracy for N = 1, cases seriousness, SVR
0.08333333333333333


## Testing accuracy (# Logistic Regression )

In [70]:
labels = None
look = []
answer_look = []
pred_look = []
file_name = '../label_process/labels/{}_seriousness_{}'
data_type = ['deaths','cases']
interval_num = ['1']
for d_type in data_type:
    for i_num in interval_num:
        with open(file_name.format(d_type,i_num),'r') as f:
            labels = csv.reader(f)
            labels = list(labels)
            if len(labels) == (len(features)+1): ## excluding header if there is
                labels = labels[1:]
        train_y,test_y,fips_train,fips_test = split_train_test(55000,56000,labels)

        tr_set = []
        tr_labels = []
        va_set = []
        va_labels = []
        total_mse = []
        svm_mse = []
        training_set = copy.deepcopy(train_x)
        training_labels = copy.deepcopy(train_y)
        tr_label_df = pd.DataFrame(train_y)
        va_label_df = pd.DataFrame(test_y)
        va_prediction = []
        svr_va_prediction = []
        va_answer = []
        for time_zone in range(int(float(i_num))):
            tr_y = list(tr_label_df.loc[:,time_zone])
            va_y = list(va_label_df.loc[:,time_zone])
            regr = LogisticRegression()
            svr_regr = SVC(kernel = 'sigmoid',gamma='auto')
            regr.fit(train_x, tr_y)
            svr_regr.fit(train_x, tr_y)
            va_prediction = va_prediction + list(regr.predict(test_x))
            svr_va_prediction = svr_va_prediction + list(svr_regr.predict(test_x))
            va_answer += va_y
        look.append(va_prediction)
        look.append(svr_va_prediction)
        answer_look.append(va_answer)
        pred_look.append(va_prediction)
        total_mse.append(accuracy_score(va_answer,va_prediction))
        svm_mse.append(accuracy_score(va_answer,svr_va_prediction))
        print('Accuracy for N = {}, {} seriousness, logistic regression'.format(i_num,d_type))
        print(sum(total_mse)/len(total_mse))
        #print('Accuracy for N = {}, {} seriousness, SVR'.format(i_num,d_type))
        #print(sum(svm_mse)/len(svm_mse))

Accuracy for N = 1, deaths seriousness, logistic regression
0.8611111111111112
Accuracy for N = 1, cases seriousness, logistic regression
0.5
